# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,20.65,87,100,10.89,PN,1696121338
1,1,san patricio,28.0170,-97.5169,30.05,67,0,5.90,US,1696121339
2,2,college,64.8569,-147.8028,10.60,46,40,3.60,US,1696121339
3,3,ushuaia,-54.8000,-68.3000,5.81,70,75,10.80,AR,1696121179
4,4,manokwari,-0.8667,134.0833,27.40,75,98,1.00,ID,1696121340


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',  # Use humidity as color
    s='Humidity',  # Use humidity as size
    cmap='Blues',       # Specify the color map
    colorbar=True,      # Show the color bar
    title='City Humidity Map',
    width=800,
    height=600
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values


# Define the conditions for filtering
temperature_condition = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)
wind_speed_condition = city_data_df['Wind Speed'] < 4.5
cloudiness_condition = city_data_df['Cloudiness'] == 0

# Combine the conditions using the '&' operator
ideal_weather_condition = temperature_condition & wind_speed_condition & cloudiness_condition

# Use the boolean mask to filter the DataFrame
ideal_weather_cities = city_data_df[ideal_weather_condition]

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,tubarjal,30.4999,38.2160,23.98,41,0,3.67,SA,1696121348
70,70,la rinconada,37.4873,-5.9826,25.03,45,0,2.06,ES,1696121362
99,99,pamandzi,-12.7967,45.2794,24.88,83,0,1.54,YT,1696121371
174,174,hamilton,39.1834,-84.5333,22.10,79,0,0.00,US,1696121186
186,186,tolanaro,-25.0319,46.9987,21.04,88,0,3.09,MG,1696121399
230,230,irymple,-34.2500,142.1667,21.72,49,0,0.45,AU,1696121416
243,243,ciutadella,40.0011,3.8414,21.45,75,0,3.61,ES,1696121419
260,260,chadron,42.8294,-102.9999,26.82,46,0,1.54,US,1696121426
262,262,luanda,-8.8368,13.2343,23.00,83,0,1.03,AO,1696121427
293,293,mableton,33.8157,-84.5619,24.26,64,0,0.00,US,1696121437


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Create a new DataFrame 'hotel_df' by copying selected columns from 'city_data_df'
hotel_df = ideal_weather_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
30,tubarjal,SA,30.4999,38.2160,41,
70,la rinconada,ES,37.4873,-5.9826,45,
99,pamandzi,YT,-12.7967,45.2794,83,
174,hamilton,US,39.1834,-84.5333,79,
186,tolanaro,MG,-25.0319,46.9987,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "api_key": geoapify_key,
    "radius": radius,
    "categories" :"accommodation.hotel"
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
   
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        #hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df





Starting hotel search
tubarjal - nearest hotel: No hotel found
la rinconada - nearest hotel: Hotel Torre de los Guzmanes
pamandzi - nearest hotel: Gite Le Madina
hamilton - nearest hotel: North Vista Manor
tolanaro - nearest hotel: Hôtel Mahavokey
irymple - nearest hotel: Chaffey Motor Inn
ciutadella - nearest hotel: El Claustre de Ciutadella
chadron - nearest hotel: Holiday Inn Express & Suites Chadron
luanda - nearest hotel: Hotel Forum
mableton - nearest hotel: Sleep Inn at Six Flags
mooloolaba - nearest hotel: Alex Seaside Resort
zaragoza - nearest hotel: Hotel Pilar Plaza
waddan - nearest hotel: No hotel found
berriane - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
craig - nearest hotel: Quality Inn & Suites
es castell - nearest hotel: Hotel Agamenon
al hazm - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
30,tubarjal,SA,30.4999,38.2160,41,No hotel found
70,la rinconada,ES,37.4873,-5.9826,45,Hotel Torre de los Guzmanes
99,pamandzi,YT,-12.7967,45.2794,83,Gite Le Madina
174,hamilton,US,39.1834,-84.5333,79,North Vista Manor
186,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
230,irymple,AU,-34.2500,142.1667,49,Chaffey Motor Inn
243,ciutadella,ES,40.0011,3.8414,75,El Claustre de Ciutadella
260,chadron,US,42.8294,-102.9999,46,Holiday Inn Express & Suites Chadron
262,luanda,AO,-8.8368,13.2343,83,Hotel Forum
293,mableton,US,33.8157,-84.5619,64,Sleep Inn at Six Flags


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
scatter_plot = ideal_weather_cities.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    s='Humidity',
    cmap='Blues',
    colorbar=True,
    title='City Humidity Map',
    width=800,
    height=600,
    hover_cols=['Hotel Name', 'Country', 'Humidity (%)']  # Customize the hover tooltip
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,Country)